### Create the repo structure and copy the TorchScript and TensorRT models into the repo

In [ ]:
!mkdir -p /repo/cls_torch/1
!mkdir -p /repo/cls_tensorrt/1

In [ ]:
!cp ./models/cls.pt /repo/cls_torch/1/model.pt
!cp ./models/cls.plan /repo/cls_tensorrt/1/model.plan

### Find the input and output names of TensorRT from ONNX model

In [ ]:
import tensorrt as trt
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(EXPLICIT_BATCH)
parser = trt.OnnxParser(network, TRT_LOGGER)
with open('./models/cls.onnx', 'rb') as model:
    parser.parse(model.read())

In [ ]:
inputs = network.get_input(0)
inputs.name, inputs.shape

In [ ]:
outputs = network.get_output(0)
outputs.name, outputs.shape

### Preparing the configs

- TorchScript

In [ ]:
%%writefile /repo/cls_torch/config.pbtxt
name: "cls_torch"
platform: "pytorch_libtorch"
max_batch_size: 32
input [
    {
      name: "INPUT__0"
      data_type: TYPE_FP32
      dims: [ 3, 256, 256 ]
    }
]
output [
    {
      name: "OUTPUT__0"
      data_type: TYPE_FP32
      dims: [ 1000 ]
    }
]
instance_group [
    {
      kind: KIND_GPU
      count: 1
    }
]

- TensorRT

In [ ]:
%%writefile /repo/cls_tensorrt/config.pbtxt
name: "cls_tensorrt"
platform: "tensorrt_plan"
max_batch_size: 32
input [
    {
      name: "input0"
      data_type: TYPE_FP32
      dims: [ 3, 256, 256 ]
    }
]
output [
    {
      name: "output0"
      data_type: TYPE_FP32
      dims: [ 1000 ]
    }
]
instance_group [
    {
      kind: KIND_GPU
      count: 1
    }
]

In [ ]:
# %%writefile /repo/cls_tensorrt/config.pbtxt
# name: "cls_tensorrt"
# platform: "tensorrt_plan"
# max_batch_size: 0
# input [
#     {
#       name: "input0"
#       data_type: TYPE_FP32
#       dims: [ -1, 3, 256, 256 ]
#     }
# ]
# output [
#     {
#       name: "output0"
#       data_type: TYPE_FP32
#       dims: [ -1, 1000 ]
#     }
# ]
# instance_group [
#     {
#       kind: KIND_GPU
#       count: 1
#     }
# ]
# dynamic_batching {
#     preferred_batch_size: [1, 32]
#     max_queue_delay_microseconds: 100
# }